## Simulation of a M/M/1 queue using processes

This simulation is adapted from the Bank Renege example in Simjulia documentation: http://simjuliajl.readthedocs.io/en/stable/examples/1_bank_renege.html

In [14]:
using SimJulia
using Distributions
using RandomStreams

Let's first simulate a fixed number of clients.

In [15]:
const RANDOM_SEED = 200
const NEW_CUSTOMERS = 5  # Total number of customers
const INTERVAL_CUSTOMERS = 2.0  # Generate new customers roughly every x seconds
const MEAN_SERVICE = 1.9

function source(env::Environment, interval::Float64, counter::Resource, limit::Float64, nserved::Array{Int64,1})
    nserved[1] = 0
    i = 0
    d = Exponential(interval)
    while (true)
        yield(Timeout(env, rand(d)))
        if (now(env) > limit) break end
        i += 1
        Process(env, new_customer, i, counter, MEAN_SERVICE, new_waits)
    end
    nserved[1] = i
end

function customer(env::Environment, idx::Int, counter::Resource, time_in_system::Float64, waits::Array{Float64,1})
    # Record the arrival time in the system
    arrive = now(env)
    println("$arrive: arrival of customer $idx")
    yield(Request(counter))
    # The simulation clock now contains the time when the client goes to the server.
    wait = now(env) - arrive
    # Record the waiting time
    waits = push!(waits, wait)
    println("$(now(env)): customer $idx has waited $wait")
    yield(Timeout(env, rand(Exponential(time_in_system))))
    println("$(now(env)): customer $idx: finished")
    yield(Release(counter))
end

customer (generic function with 2 methods)

In [16]:
# Setup and start the simulation
println("M/M/1 with processes")
waits = Array(Float64, NEW_CUSTOMERS)
srand(RANDOM_SEED)
env = Environment()

M/M/1 with processes


SimJulia.Environment(0.0,Base.Collections.PriorityQueue{SimJulia.BaseEvent,SimJulia.EventKey,Base.Order.ForwardOrdering}(),0,0,Nullable{SimJulia.Process}())

In [18]:
nserved = [ 0 ]
srand(RANDOM_SEED)
env = Environment()
counter = Resource(env, 1)
new_waits = Float64[]
Process(env, new_source!, INTERVAL_CUSTOMERS, counter, 5.0, nserved)
run(env)

1.2050403125807367: arrival of customer 1
1.2050403125807367: customer 1 has waited 0.0
1.4181503510677775: arrival of customer 2
1.7579446126823792: arrival of customer 3
3.3655840436959545: arrival of customer 4
3.5932863161203605: customer 1: finished
3.5932863161203605: customer 2 has waited 2.1751359650525828
3.6479776093878162: customer 2: finished
3.6479776093878162: customer 3 has waited 1.890032996705437
4.429115262189869: customer 3: finished
4.429115262189869: customer 4 has waited 1.0635312184939147
11.115163310168295: customer 4: finished
